In [1]:
import sys
sys.version 

'3.10.14 | packaged by Anaconda, Inc. | (main, May  6 2024, 19:44:50) [MSC v.1916 64 bit (AMD64)]'

In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [3]:
import os
import re
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.tools import Tool

gpt4o = ChatOpenAI(model= "gpt-4o", api_key=os.getenv("OPENAI_API_KEY"), temperature=0)
gpt3_0 = ChatOpenAI(model= "gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"), temperature=0)

In [4]:
from typing import Dict
def analyze_code_complexity(code: str) -> Dict[str, int]:
    tree = ast.parse(code)
    return {
        'num_functions': len([node for node in ast.walk(tree) if isinstance(node, ast.FunctionDef)]),
        'num_classes': len([node for node in ast.walk(tree) if isinstance(node, ast.ClassDef)]),
        'num_loops': len([node for node in ast.walk(tree) if isinstance(node, (ast.For, ast.While))]),
        'num_conditionals': len([node for node in ast.walk(tree) if isinstance(node, ast.If)])
    }

In [5]:
def _sanitize_output(text: str):
    _, after = text.split("```python")
    return after.split("```")[0].strip()

In [6]:
class CustomStrOutputParser(StrOutputParser):
    def parse(self, output: str):
        print(output)
        return super().parse(output)

In [7]:
from typing import Optional, Callable

def create_prompt_chain(
    system_template: str, 
    human_template: str,
    model: Optional[ChatOpenAI] = None) -> Callable:
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_template), 
        ("human", human_template)
    ])
    return prompt | model | CustomStrOutputParser()

## 사용자 입력 문자열 분리

In [8]:
def parse_user_input(user_input: str): 
    input_parser_template = """
    Analyze the following user input and extract the following components:
    1. The coding test problem description
    2. The user's Python code
    3. The input data for testing the code
    4. Analyze the time complexity and space complexity of the user's Python code
    
    User Input:
    {user_input}
    
    Please format your response as follows:
    Problem Description: [Extracted problem description]
    
    User Code:
    ```python
    [Extracted user code]
    ```
    
    Input Data: [Extracted input data for testing] 
    
    Time Complexity: [Time complexity analysis] 
    
    Space Complexity: [Space complexity analysis] 
    """
    separation_chain = create_prompt_chain(input_parser_template, "{user_input}", gpt3_0)
    
    # LLM을 사용하여 사용자 입력을 분석하고 각 부분을 추출
    parsed_response = separation_chain.invoke({"user_input": user_input})

     # 정규 표현식을 사용하여 각 부분 추출
    problem_description_match = re.search(r'Problem Description:\s*(.*?)\s*User Code:', parsed_response, re.DOTALL)
    user_code_match = _sanitize_output(parsed_response)
    input_data_match = re.search(r'Input Data:\s*(.*?)\s*Time Complexity:', parsed_response, re.DOTALL)
    time_complexity_match = re.search(r'Time Complexity:\s*(.*?)\s*Space Complexity:', parsed_response, re.DOTALL)
    space_complexity_match = re.search(r'Space Complexity:\s*(.*)', parsed_response, re.DOTALL)
    
    # 각 부분이 발견되면 추출, 그렇지 않으면 빈 문자열 반환
    problem_description = problem_description_match.group(1).strip() if problem_description_match else ""
    user_code = user_code_match.strip() if user_code_match else ""
    input_data = input_data_match.group(1).strip() if input_data_match else ""
    time_complexity = time_complexity_match.group(1).strip() if time_complexity_match else ""
    space_complexity = space_complexity_match.group(1).strip() if space_complexity_match else ""
    
    return problem_description, user_code, input_data, time_complexity, space_complexity

In [9]:
user_input = """
I need help with this coding problem: Leetcode Problem 200 - Number of Islands.
Given an m x n 2D binary grid grid which represents a map of '1's (land) and '0's (water), return the number of islands. An island is surrounded by water and is formed by connecting adjacent lands horizontally or vertically. You may assume all four edges of the grid are all surrounded by water.

Here's my code:
def numIslands(grid):
    if not grid:
        return 0
 
    rows, cols = len(grid), len(grid[0])
    num_islands = 0
 
    # DFS function to traverse the island
    def dfs(r, c):
        if r < 0 or c < 0 or r >= rows or c >= cols or grid[r][c] == '0':
            return
        grid[r][c] = '0'  # Mark the cell as visited
        # Visit all four directions
        dfs(r-1, c)
        dfs(r+1, c)
        dfs(r, c-1)
        dfs(r, c+1)
 
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == '1':
                dfs(r, c)
                num_islands += 1
 
    return num_islands

Can you review my code and test it with this input: 
grid = [
  ["1","1","0","0","0"],
  ["1","1","0","0","0"],
  ["0","0","1","0","0"],
  ["0","0","0","1","1"]
]
"""

In [10]:
problem_description, user_code, input_data, time_complexity, space_complexity = parse_user_input(user_input)

Problem Description: Given a 2D binary grid representing a map of '1's (land) and '0's (water), the task is to return the number of islands. An island is surrounded by water and is formed by connecting adjacent lands horizontally or vertically. All four edges of the grid are assumed to be surrounded by water.

User Code:
```python
def numIslands(grid):
    if not grid:
        return 0
 
    rows, cols = len(grid), len(grid[0])
    num_islands = 0
 
    # DFS function to traverse the island
    def dfs(r, c):
        if r < 0 or c < 0 or r >= rows or c >= cols or grid[r][c] == '0':
            return
        grid[r][c] = '0'  # Mark the cell as visited
        # Visit all four directions
        dfs(r-1, c)
        dfs(r+1, c)
        dfs(r, c-1)
        dfs(r, c+1)
 
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == '1':
                dfs(r, c)
                num_islands += 1
 
    return num_islands
```

Input Data: 
grid = [
  ["1","1","0","0","

## exec 함수 형태로 만들기 (사용자 코드와 입력 데이터 하드코딩)

In [11]:
def modify_user_code_for_execution(user_code: str, input_data: str) -> str: 
    # 입력 데이터 하드코딩 및 함수 호출 추가 템플릿
    code_execution_template = """
    You are provided with a Python code and input data. Your task is to prepare the code so that it can be executed directly using the exec function in Python.
    
    Your task is to:
    1. Ensure that the input data is correctly hardcoded into the user's code in the appropriate places.
    2. If the code contains any functions (e.g., a main function or any other defined functions), add the necessary function calls at the bottom if they are missing.
    3. Do not change the structure or logic of the user's code.
    4. Ensure that after modification, the code is ready to be executed directly with the exec function.
    
    User Code:
    ```python
    {user_code}
    ```
    
    Input Data: {input_data}
    
    Please return the modified Python code that can be executed directly using exec. """
    
    execution_chain = create_prompt_chain(code_execution_template, "{user_code}\n{input_data}", gpt4o)
    
    modified_code_response = execution_chain.invoke({"user_code": user_code, "input_data": input_data})
    modified_code = _sanitize_output(modified_code_response)
    
    return modified_code

In [12]:
exec_code = modify_user_code_for_execution(user_code, input_data)

Here is the modified Python code that can be executed directly using the `exec` function:

```python
def numIslands(grid):
    if not grid:
        return 0
 
    rows, cols = len(grid), len(grid[0])
    num_islands = 0
 
    # DFS function to traverse the island
    def dfs(r, c):
        if r < 0 or c < 0 or r >= rows or c >= cols or grid[r][c] == '0':
            return
        grid[r][c] = '0'  # Mark the cell as visited
        # Visit all four directions
        dfs(r-1, c)
        dfs(r+1, c)
        dfs(r, c-1)
        dfs(r, c+1)
 
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == '1':
                dfs(r, c)
                num_islands += 1
 
    return num_islands

grid = [
  ["1","1","0","0","0"],
  ["1","1","0","0","0"],
  ["0","0","1","0","0"],
  ["0","0","0","1","1"]
]

print(numIslands(grid))
```

This code includes the input data and a call to the `numIslands` function, followed by a `print` statement to display the result. You can 

In [13]:
print(exec_code)

def numIslands(grid):
    if not grid:
        return 0
 
    rows, cols = len(grid), len(grid[0])
    num_islands = 0
 
    # DFS function to traverse the island
    def dfs(r, c):
        if r < 0 or c < 0 or r >= rows or c >= cols or grid[r][c] == '0':
            return
        grid[r][c] = '0'  # Mark the cell as visited
        # Visit all four directions
        dfs(r-1, c)
        dfs(r+1, c)
        dfs(r, c-1)
        dfs(r, c+1)
 
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == '1':
                dfs(r, c)
                num_islands += 1
 
    return num_islands

grid = [
  ["1","1","0","0","0"],
  ["1","1","0","0","0"],
  ["0","0","1","0","0"],
  ["0","0","0","1","1"]
]

print(numIslands(grid))


## memory_profiler 함수 (하드코딩 완료된 함수에 @profile 추가)

In [24]:
def modify_code_with_memory_profiler(exec_code: str) -> str:
    # memory_profiler 데코레이터 추가 템플릿
    memory_profiler_template = """
    You are provided with a Python code. Your task is to modify the code so that the memory usage of the code can be profiled using the memory_profiler library.
    
    Your task is to:
    1. Add the necessary imports for memory profiling (from memory_profiler import profile).
    2. Add the @profile decorator above the main function or relevant functions that should be profiled for memory usage.
    3. Ensure that the original structure and logic of the user's code remain unchanged.
    4. If there is no function execution (such as 'main()' or any other function calls), add the appropriate function call at the end of the code so that the function decorated with @profile is executed. This is important because the @profile decorator must be triggered for memory usage to be tracked.
    
    Additionally, keep in mind that the modified code will be executed using a subprocess. Ensure that the code can be executed in this environment without errors.
    
    User Code:
    ```python
    {user_code}
    ```
    
    Please return the modified Python code with @profile decorator added, memory_profiler set up, and any missing function calls appropriately added. 
    """

    memory_chain = create_prompt_chain(memory_profiler_template, "{user_code}", gpt4o)
    modified_code_response = memory_chain.invoke({"user_code": exec_code})
    modified_code = _sanitize_output(modified_code_response)
    
    return modified_code

In [25]:
memory_code = modify_code_with_memory_profiler(exec_code)

Here is the modified code with the necessary imports and the `@profile` decorator added. The `numIslands` function is decorated to profile its memory usage, and the function call is included to ensure the profiling is triggered.

```python
from memory_profiler import profile

@profile
def numIslands(grid):
    if not grid:
        return 0
 
    rows, cols = len(grid), len(grid[0])
    num_islands = 0
 
    # DFS function to traverse the island
    def dfs(r, c):
        if r < 0 or c < 0 or r >= rows or c >= cols or grid[r][c] == '0':
            return
        grid[r][c] = '0'  # Mark the cell as visited
        # Visit all four directions
        dfs(r-1, c)
        dfs(r+1, c)
        dfs(r, c-1)
        dfs(r, c+1)
 
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == '1':
                dfs(r, c)
                num_islands += 1
 
    return num_islands

grid = [
  ["1","1","0","0","0"],
  ["1","1","0","0","0"],
  ["0","0","1","0","0"],
  ["0","0

In [26]:
print(memory_code)

from memory_profiler import profile

@profile
def numIslands(grid):
    if not grid:
        return 0
 
    rows, cols = len(grid), len(grid[0])
    num_islands = 0
 
    # DFS function to traverse the island
    def dfs(r, c):
        if r < 0 or c < 0 or r >= rows or c >= cols or grid[r][c] == '0':
            return
        grid[r][c] = '0'  # Mark the cell as visited
        # Visit all four directions
        dfs(r-1, c)
        dfs(r+1, c)
        dfs(r, c-1)
        dfs(r, c+1)
 
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == '1':
                dfs(r, c)
                num_islands += 1
 
    return num_islands

grid = [
  ["1","1","0","0","0"],
  ["1","1","0","0","0"],
  ["0","0","1","0","0"],
  ["0","0","0","1","1"]
]

print(numIslands(grid))


## 코드 프로파일링

In [27]:
import cProfile
import pstats
import io
import timeit
import tracemalloc
import tempfile
import subprocess

def analyze_code(exec_code: str, memory_code: str):
    exec_time = 0
    memory_usage = {'current_memory': 0, 'peak_memory': 0}
    profiling_result = ""
    memory_profile_output = "" # 라인별 메모리 사용량 측정 (외부 프로세스에서)
    error_status = "no_error"
        
    # cProfile을 사용한 성능 프로파일링
    profiler = cProfile.Profile()
    
    try:
        tracemalloc.start()  # 메모리 사용량 측정 시작
        start_time = timeit.default_timer()  # 시간 측정 시작
        profiler.enable()  # 성능 프로파일링 시작
        
        exec(exec_code, globals())  # exec로 코드 실행
        
        # 프로파일링 도구 종료
        profiler.disable()  # 성능 프로파일링 종료
        end_time = timeit.default_timer()  # 시간 측정 종료
        current, peak = tracemalloc.get_traced_memory()  # 메모리 사용량 측정 종료
        tracemalloc.stop()  # 메모리 추적 종료

        # 실행 시간 및 메모리 사용량 저장
        exec_time = end_time - start_time
        memory_usage = {
            'current_memory': current / 10**6,  # MB 단위
            'peak_memory': peak / 10**6         # MB 단위
        }
        
        # profiler.print_stats() 결과를 변수에 담기
        s = io.StringIO()
        ps = pstats.Stats(profiler, stream=s)
        ps.print_stats()
        profiling_result = s.getvalue()

            
    except Exception:
        error_status = f"exec_error"
    

    # memory_code를 위한 임시 파일 생성 (subprocess에서 메모리 프로파일링할 코드)
    with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False, encoding='utf-8') as temp_file_memory:
        temp_file_name_memory = temp_file_memory.name
        temp_file_memory.write(memory_code)

    # memory_profiler를 사용하여 라인별 메모리 사용량 측정
    try:
        command = f"python -m memory_profiler {temp_file_name_memory}"
        result = subprocess.run(command, shell=True, capture_output=True, text=True, check=True)
        
        # 라인별 메모리 사용량만 출력
        memory_profile_output = result.stdout

    except subprocess.CalledProcessError:
        memory_profile_output = f"subprocess_error"
    
    finally:
        # 임시 파일 삭제
        os.remove(temp_file_name_memory)
        
    return exec_time, memory_usage, profiling_result, memory_profile_output, error_status

In [28]:
# analyze_code 호출
exec_time, memory_usage, profiling_result, memory_profile_output, error_status = analyze_code(exec_code, memory_code)

3


In [29]:
# 각 결과를 print로 출력
print("Execution Time:", exec_time)
print("Memory Usage:")
print(f"  Current Memory: {memory_usage['current_memory']} MB")
print(f"  Peak Memory: {memory_usage['peak_memory']} MB")
print("Profiling Result:\n", profiling_result)
print("Memory Profile Output:\n", memory_profile_output)
print("Error Status:", error_status)

Execution Time: 0.0014325999654829502
Memory Usage:
  Current Memory: 0.009012 MB
  Peak Memory: 0.09978 MB
Profiling Result:
          70 function calls (42 primitive calls) in 0.001 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 C:\Users\user\miniconda3\envs\test1\lib\threading.py:553(is_set)
        1    0.000    0.000    0.000    0.000 C:\Users\user\miniconda3\envs\test1\lib\threading.py:1102(_wait_for_tstate_lock)
        1    0.000    0.000    0.000    0.000 C:\Users\user\miniconda3\envs\test1\lib\threading.py:1169(is_alive)
        1    0.000    0.000    0.000    0.000 C:\Users\user\miniconda3\envs\test1\lib\site-packages\zmq\sugar\socket.py:626(send)
        1    0.000    0.000    0.000    0.000 C:\Users\user\miniconda3\envs\test1\lib\site-packages\dateutil\tz\tz.py:74(utcoffset)
        1    0.000    0.000    0.000    0.000 C:\Users\user\miniconda3\envs\test1\lib\s

## 최종 코드 리뷰

In [35]:
def generate_code_analysis(
    user_code: str, 
    time_complexity: str, 
    space_complexity: str, 
    exec_time: float, 
    memory_usage: dict,  # memory_usage는 dict 타입
    exec_code: str, 
    memory_code: str, 
    profiling_result: str, 
    memory_profile_output: str
) -> str:
    analyze_code_systemTemplate = """You are an expert code analyzer and performance optimization specialist. 
    Your task is to analyze the given code performance metrics and provide insights and optimization suggestions.
    
    Consider the following aspects in your analysis:
    1. Time complexity
    2. Space complexity
    3. Execution time
    4. Memory usage
    5. Profiling results
    6. Memory profile output
    
    Provide a comprehensive analysis and suggest optimizations if necessary.
    """
    
    analyze_code_humanTemplate = """Please analyze the following code performance metrics:
    
    User Code:
    ```python
    {user_code}
    ```
      
    Time Complexity: {time_complexity}
    Space Complexity: {space_complexity}
    Execution Time: {exec_time} seconds
    Memory Usage: Current - {current_memory} MB, Peak - {peak_memory} MB
    
    Execution Code:
    {exec_code}
    
    Memory Code:
    {memory_code}
    
    Profiling Result:
    {profiling_result}
    
    Memory Profile Output:
    {memory_profile_output}
    
    Based on these metrics, provide an analysis of the code's performance and suggest optimizations if necessary.
    """

    input_state_data = {
        "user_code": user_code,
        "time_complexity": time_complexity,
        "space_complexity": space_complexity,
        "exec_time": exec_time,
        "current_memory": memory_usage.get("current_memory", "N/A"),  # 현재 메모리 사용량
        "peak_memory": memory_usage.get("peak_memory", "N/A"),        # 최고 메모리 사용량
        "exec_code": exec_code,
        "memory_code": memory_code,
        "profiling_result": profiling_result,
        "memory_profile_output": memory_profile_output
    }

    code_analysis_chain = create_prompt_chain(analyze_code_systemTemplate, analyze_code_humanTemplate, gpt4o)
    code_analysis_response = code_analysis_chain.invoke(input_state_data)

    return code_analysis_response    

In [36]:
result = generate_code_analysis(user_code, 
    time_complexity, 
    space_complexity, 
    exec_time, 
    memory_usage, 
    exec_code, 
    memory_code, 
    profiling_result, 
    memory_profile_output)

### Analysis of Code Performance

#### 1. Time Complexity
The time complexity of the provided code is \(O(m \times n)\), where \(m\) is the number of rows and \(n\) is the number of columns in the grid. This is because the code performs a depth-first search (DFS) on each cell of the grid once. Each cell is visited once and marked as visited, ensuring that no cell is processed more than once.

#### 2. Space Complexity
The space complexity of the code is \(O(m \times n)\) due to the recursive calls in the DFS function. In the worst case, the recursion stack can go as deep as the number of cells in the grid, especially if the grid is filled with '1's.

#### 3. Execution Time
The execution time is reported as 0.0014325999654829502 seconds, which is quite efficient for the given input size. This indicates that the algorithm performs well within acceptable limits for small to moderately sized grids.

#### 4. Memory Usage
- **Current Memory Usage:** 0.009012 MB
- **Peak Memory Usage:** 0.0997

In [37]:
from IPython.display import display, Markdown
display(Markdown(result))

### Analysis of Code Performance

#### 1. Time Complexity
The time complexity of the provided code is \(O(m \times n)\), where \(m\) is the number of rows and \(n\) is the number of columns in the grid. This is because the code performs a depth-first search (DFS) on each cell of the grid once. Each cell is visited once and marked as visited, ensuring that no cell is processed more than once.

#### 2. Space Complexity
The space complexity of the code is \(O(m \times n)\) due to the recursive calls in the DFS function. In the worst case, the recursion stack can go as deep as the number of cells in the grid, especially if the grid is filled with '1's.

#### 3. Execution Time
The execution time is reported as 0.0014325999654829502 seconds, which is quite efficient for the given input size. This indicates that the algorithm performs well within acceptable limits for small to moderately sized grids.

#### 4. Memory Usage
- **Current Memory Usage:** 0.009012 MB
- **Peak Memory Usage:** 0.09978 MB

The memory usage is relatively low, indicating that the algorithm is efficient in terms of memory consumption for the given input size.

#### 5. Profiling Results
The profiling results show that the function `numIslands` and its nested `dfs` function are called multiple times, but the total time spent in these calls is minimal. The majority of the time is spent in the built-in methods and functions, which are optimized for performance.

#### 6. Memory Profile Output
The memory profile output indicates that the memory usage remains constant throughout the execution of the function. There are no significant memory spikes, and the memory usage is stable at 41.3 MiB.

### Optimization Suggestions

1. **Iterative DFS/BFS:**
   - **Reason:** Recursive DFS can lead to a stack overflow for very large grids due to deep recursion. An iterative approach using a stack (for DFS) or a queue (for BFS) can mitigate this issue.
   - **Implementation:**
     ```python
     def numIslands(grid):
         if not grid:
             return 0

         rows, cols = len(grid), len(grid[0])
         num_islands = 0

         def bfs(r, c):
             queue = [(r, c)]
             while queue:
                 x, y = queue.pop(0)
                 if x < 0 or y < 0 or x >= rows or y >= cols or grid[x][y] == '0':
                     continue
                 grid[x][y] = '0'
                 queue.append((x-1, y))
                 queue.append((x+1, y))
                 queue.append((x, y-1))
                 queue.append((x, y+1))

         for r in range(rows):
             for c in range(cols):
                 if grid[r][c] == '1':
                     bfs(r, c)
                     num_islands += 1

         return num_islands
     ```

2. **In-place Modification:**
   - **Reason:** The current implementation modifies the input grid in place, which is efficient in terms of space. However, if the input grid should not be modified, a copy of the grid can be used.
   - **Implementation:**
     ```python
     import copy

     def numIslands(grid):
         if not grid:
             return 0

         grid = copy.deepcopy(grid)
         rows, cols = len(grid), len(grid[0])
         num_islands = 0

         def dfs(r, c):
             if r < 0 or c < 0 or r >= rows or c >= cols or grid[r][c] == '0':
                 return
             grid[r][c] = '0'
             dfs(r-1, c)
             dfs(r+1, c)
             dfs(r, c-1)
             dfs(r, c+1)

         for r in range(rows):
             for c in range(cols):
                 if grid[r][c] == '1':
                     dfs(r, c)
                     num_islands += 1

         return num_islands
     ```

3. **Early Termination:**
   - **Reason:** If the grid is very sparse (contains many '0's), early termination can be implemented to skip unnecessary checks.
   - **Implementation:** This is more of a heuristic and may not always be applicable.

### Conclusion
The provided code is efficient in terms of both time and space complexity for the given input size. The execution time and memory usage are within acceptable limits. However, for very large grids, an iterative approach to DFS/BFS can be more robust and prevent stack overflow issues. Additionally, if the input grid should not be modified, a deep copy can be used to preserve the original grid.